Python 是支持面向对象的，很多情况下使用面向对象编程会使得代码更加容易扩展，并且可维护性更高，但是如果你写的多了或者某一对象非常复杂了，其中的一些写法会相当相当繁琐，而且我们会经常碰到对象和 JSON 序列化及反序列化的问题，原生的 Python 转起来还是很费劲的。

可能这么说大家会觉得有点抽象，那么这里举几个例子来感受一下。

首先让我们定义一个对象吧，比如颜色。我们常用 RGB 三个原色来表示颜色，R、G、B 分别代表红、绿、蓝三个颜色的数值，范围是 0-255，也就是每个原色有 256 个取值。如 RGB(0, 0, 0) 就代表黑色，RGB(255, 255, 255) 就代表白色，RGB(255, 0, 0) 就代表红色，如果不太明白可以具体看看 RGB 颜色的定义哈。

好，那么我们现在如果想定义一个颜色对象，那么正常的写法就是这样了，创建这个对象的时候需要三个参数，就是 R、G、B 三个数值，定义如下：

In [2]:
class Color:
    """
    Color Object of RGB
    """
    def __init__(self, r, g, b):
        self.r = r
        self.g = g
        self.b = b

In [3]:
color = Color(255, 255, 255)
print(color)

我们知道，在 Python 里面想要定义某个对象本身的打印输出结果的时候，需要实现它的 __repr__ 方法，所以我们比如我们添加这么一个方法：

```python
def __repr__(self):
    return f'{self.__class__.__name__}(r={self.r}, g={self.g}, b={self.b})'
```

In [4]:
class Color:
    """
    Color Object of RGB
    """
    def __init__(self, r, g, b):
        self.r = r
        self.g = g
        self.b = b
        
    def __repr__(self):
        return f'{self.__class__.__name__}(r={self.r}, g={self.g}, b={self.b})'

In [5]:
color = Color(255, 255, 255)
print(color)

Color(r=255, g=255, b=255)


再继续，如果我们要想实现这个对象里面的 __eq__、__lt__ 等各种方法来实现对象之间的比较呢？照样需要继续定义成类似这样子的形式：
```python
def __lt__(self, other):
    if not isinstance(other, self.__class__): return NotImplemented
    return (self.r, self.g, self.b) < (other.r, other.g, other.b)

```

In [9]:
class Color:
    """
    Color Object of RGB
    """
    def __init__(self, r, g, b):
        self.r = r
        self.g = g
        self.b = b
        
    def __repr__(self):
        return f'{self.__class__.__name__}(r={self.r}, g={self.g}, b={self.b})'
    
    def __lt__(self, other):
        if not isinstance(other, self.__class__): return NotImplemented
        return (self.r, self.g, self.b) < (other.r, other.g, other.b)

In [10]:
c1 = Color(1, 255, 255)
c2 = Color(255
           , 255, 255)

c1<c2


True

首先我们来介绍下 attrs 这个库，其官方的介绍如下：

    attrs 是这样的一个 Python 工具包，它能将你从繁综复杂的实现上解脱出来，享受编写 Python 类的快乐。它的目标就是在不减慢你编程速度的前提下，帮助你来编写简洁而又正确的代码。

其实意思就是用了它，定义和实现 Python 类变得更加简洁和高效。

首先明确一点，我们现在是装了 attrs 和 cattrs 这两个库，但是实际导入的时候是使用 attr 和 cattr 这两个包，是不带 s 的。

在 attr 这个库里面有两个比较常用的组件叫做 attrs 和 attr，前者是主要用来修饰一个自定义类的，后者是定义类里面的一个字段的。有了它们，我们就可以将上文中的定义改写成下面的样子：

In [13]:
from attr import attrs, attrib

@attrs
class Color(object):
    r = attrib(type=int, default=0)
    g = attrib(type=int, default=0)
    b = attrib(type=int, default=0)

if __name__ == '__main__':
    color = Color(255, 255, 255)
    print(color)

Color(r=255, g=255, b=255)


观察一下有什么变化，是不是变得更简洁了？r、g、b 三个属性都只写了一次，同时还指定了各个字段的类型和默认值，另外也不需要再定义 __init__ 方法和 __repr__ 方法了，一切都显得那么简洁。一个字，爽！

实际上，主要是 attrs 这个修饰符起了作用，然后根据定义的 attrib 属性自动帮我们实现了 __init__、__repr__、__eq__、__ne__、__lt__、__le__、__gt__、__ge__、__hash__ 这几个方法。

如使用 attrs 修饰的类定义是这样子：

```python
from attr import attrs, attrib

@attrs
class SmartClass(object):
    a = attrib()
    b = attrib()
```
其实就相当于已经实现了这些方法：

```python
class RoughClass(object):
    def __init__(self, a, b):
        self.a = a
        self.b = b

    def __repr__(self):
        return "RoughClass(a={}, b={})".format(self.a, self.b)

    def __eq__(self, other):
        if other.__class__ is self.__class__:
            return (self.a, self.b) == (other.a, other.b)
        else:
            return NotImplemented

    def __ne__(self, other):
        result = self.__eq__(other)
        if result is NotImplemented:
            return NotImplemented
        else:
            return not result

    def __lt__(self, other):
        if other.__class__ is self.__class__:
            return (self.a, self.b) < (other.a, other.b)
        else:
            return NotImplemented

    def __le__(self, other):
        if other.__class__ is self.__class__:
            return (self.a, self.b) <= (other.a, other.b)
        else:
            return NotImplemented

    def __gt__(self, other):
        if other.__class__ is self.__class__:
            return (self.a, self.b) > (other.a, other.b)
        else:
            return NotImplemented

    def __ge__(self, other):
        if other.__class__ is self.__class__:
            return (self.a, self.b) >= (other.a, other.b)
        else:
            return NotImplemented

    def __hash__(self):
        return hash((self.__class__, self.a, self.b))

```



首先库的名字就叫做 attrs，这个就是装 Python 包的时候这么装就行了。但是库的名字和导入的包的名字确实是不一样的，我们用的时候就导入 attr 这个包就行了，里面包含了各种各样的模块和组件，这是完全固定的。

好，然后接下来看看 attr 包里面包含了什么，刚才我们引入了 attrs 和 attrib。

首先是 attrs，它主要是用来修饰 class 类的，而 attrib 主要是用来做属性定义的，这个就记住它们两个的用法就好了。

翻了一下源代码，发现其实它还有一些别名：

```python
s = attributes = attrs
ib = attr = attrib

```
也就是说，attrs 可以用 s 或 attributes 来代替，attrib 可以用 attr 或 ib 来代替。

既然是别名，那么上面的类就可以改写成下面的样子：

In [18]:
from attr import s, ib

@s
class Color(object):
    r = ib(type=int, default=0)
    g = ib(type=int, default=0)
    b = ib(type=int, default=0)

if __name__ == '__main__':
    color = Color(255, 255, 255)
    print(color)

Color(r=255, g=255, b=255)


不是更加简洁了，当然你也可以把 s 改写为 attributes，ib 改写为 attr，随你怎么用啦。

不过我觉得比较舒服的是 attrs 和 attrib 的搭配，感觉可读性更好一些，当然这个看个人喜好。

所以总结一下：

    - 库名：attrs

    - 导入包名：attr

    - 修饰类：s 或 attributes 或 attrs

    - 定义属性：ib 或 attr 或 attrib

OK，理清了这几部分内容，我们继续往下深入了解它的用法吧。

In [20]:
from attr import attrs, attrib

@attrs
class Point(object):
    x = attrib()
    y = attrib()

其中 attrib 里面什么参数都没有，如果我们要使用的话，参数可以顺次指定，也可以根据名字指定，如：

In [21]:
p1 = Point(1, 2)
print(p1)

Point(x=1, y=2)


In [22]:
p2 = Point(x=1, y=2)
print(p2)

Point(x=1, y=2)


OK，接下来让我们再验证下类之间的比较方法，由于使用了 attrs，相当于我们定义的类已经有了 __eq__、__ne__、__lt__、__le__、__gt__、__ge__ 这几个方法，所以我们可以直接使用比较符来对类和类之间进行比较，下面我们用实例来感受一下：

In [23]:
print('Equal:', Point(1, 2) == Point(1, 2))
print('Not Equal(ne):', Point(1, 2) != Point(3, 4))
print('Less Than(lt):', Point(1, 2) < Point(3, 4))
print('Less or Equal(le):', Point(1, 2) <= Point(1, 4), Point(1, 2) <= Point(1, 2))
print('Greater Than(gt):', Point(4, 2) > Point(3, 2), Point(4, 2) > Point(3, 1))
print('Greater or Equal(ge):', Point(4, 2) >= Point(4, 1))

Equal: True
Not Equal(ne): True
Less Than(lt): True
Less or Equal(le): True True
Greater Than(gt): True True
Greater or Equal(ge): True


可能有的朋友不知道 ne、lt、le 什么的是什么意思，不过看到这里你应该明白啦，ne 就是 Not Equal 的意思，就是不相等，le 就是 Less or Equal 的意思，就是小于或等于。

其内部怎么实现的呢，就是把类的各个属性转成元组来比较了，比如 Point(1, 2) < Point(3, 4) 实际上就是比较了 (1, 2) 和 (3, 4) 两个元组，那么元组之间的比较逻辑又是怎样的呢，这里就不展开了，如果不明白的话可以参考官方文档：https://docs.python.org/3/library/stdtypes.html#comparisons。

## 属性定义

现在看来，对于这个类的定义莫过于每个属性的定义了，也就是 attrib 的定义。对于 attrib 的定义，我们可以传入各种参数，不同的参数对于这个类的定义有非常大的影响。

下面我们就来详细了解一下每个属性的具体参数和用法吧。

首先让我们概览一下总共可能有多少可以控制一个属性的参数，我们用 attrs 里面的 fields 方法可以查看一下：

In [24]:
from attr import attrs, attrib, fields

@attrs
class Point(object):
    x = attrib()
    y = attrib()

print(fields(Point))

(Attribute(name='x', default=NOTHING, validator=None, repr=True, cmp=True, hash=None, init=True, metadata=mappingproxy({}), type=None, converter=None, kw_only=False), Attribute(name='y', default=NOTHING, validator=None, repr=True, cmp=True, hash=None, init=True, metadata=mappingproxy({}), type=None, converter=None, kw_only=False))


    name：属性的名字，是一个字符串类型。

    default：属性的默认值，如果没有传入初始化数据，那么就会使用默认值。如果没有默认值定义，那么就是 NOTHING，即没有默认值。

    validator：验证器，检查传入的参数是否合法。

    init：是否参与初始化，如果为 False，那么这个参数不能当做类的初始化参数，默认是 True。

    metadata：元数据，只读性的附加数据。

    type：类型，比如 int、str 等各种类型，默认为 None。

    converter：转换器，进行一些值的处理和转换器，增加容错性。

    kw_only：是否为强制关键字参数，默认为 False。

## 属性名

x = attrib()

那么其属性名就是 x。

## 默认值

对于默认值，如果在初始化的时候没有指定，那么就会默认使用默认值进行初始化，我们看下面的一个实例：

In [25]:
from attr import attrs, attrib, fields

@attrs
class Point(object):
    x = attrib()
    y = attrib(default=100)

if __name__ == '__main__':
    print(Point(x=1, y=3))
    print(Point(x=1))

Point(x=1, y=3)
Point(x=1, y=100)


在这里我们将 y 属性的默认值设置为了 100，在初始化的时候，第一次都传入了 x、y 两个参数，第二次只传入了 x 这个参数，看下运行结果：

## 初始化

如果一个类的某些属性不想参与初始化，比如想直接设置一个初始值，一直固定不变，我们可以将属性的 init 参数设置为 False，看一个实例：

In [26]:
from attr import attrs, attrib

@attrs
class Point(object):
    x = attrib(init=False, default=10)
    y = attrib()

if __name__ == '__main__':
    print(Point(3))

Point(x=10, y=3)


没什么问题，y 被赋值为了我们设置的值 3。

那假如我们非要设置 x 呢？会发生什么，比如改写成这样子：

In [27]:
Point(1, 2)

TypeError: __init__() takes 2 positional arguments but 3 were given

## 强制关键字

强制关键字是 Python 里面的一个特性，在传入的时候必须使用关键字的名字来传入，如果不太理解可以再了解下 Python 的基础。

设置了强制关键字参数的属性必须要放在后面，其后面不能再有非强制关键字参数的属性，否则会报这样的错误：

我们来看一个例子，我们将最后一个属性设置 kw_only 参数为 True：

In [28]:
from attr import attrs, attrib, fields

@attrs
class Point(object):
    x = attrib(default=0)
    y = attrib(kw_only=True)

if __name__ == '__main__':
    print(Point(1, y=3))

Point(x=1, y=3)


如果没有指定 y 这个名字，像这样调用：
```
Point(1, 3)
```
那么就会报错：

```python
TypeError: __init__() takes from 1 to 2 positional arguments but 3 were given
```


所以，这个参数就是设置初始化传参必须要用名字来传，否则会出现错误。

注意，如果我们将一个属性设置了 init 为 False，那么 kw_only 这个参数会被忽略。

## 验证器

有时候在设置一个属性的时候必须要满足某个条件，比如性别必须要是男或者女，否则就不合法。对于这种情况，我们就需要有条件来控制某些属性不能为非法值。下面我们看一个实例：

In [32]:
from attr import attrs, attrib

def is_valid_gender(instance, attribute, value):
    if value not in ['male', 'female']:
        raise ValueError(f'gender {value} is not valid')

@attrs
class Person(object):
    name = attrib()
    gender = attrib(validator=is_valid_gender)

if __name__ == '__main__':
    print(Person(name='Mike', gender='male'))
    # ValueError 
    # print(Person(name='Mike', gender='mlae'))  # ValueError

Person(name='Mike', gender='male')


在这里我们定义了一个验证器 Validator 方法，叫做 is_valid_gender。然后定义了一个类 Person 还有它的两个属性 name 和 gender，其中 gender 定义的时候传入了一个参数 validator，其值就是我们定义的 Validator 方法。

这个 Validator 定义的时候有几个固定的参数：

    instance：类对象

    attribute：属性名

    value：属性值

这是三个参数是固定的，在类初始化的时候，其内部会将这三个参数传递给这个 Validator，因此 Validator 里面就可以接受到这三个值，然后进行判断即可。在 Validator 里面，我们判断如果不是男性或女性，那么就直接抛出错误。

下面做了两个实验，一个就是正常传入 male，另一个写错了，写的是 mlae，观察下运行结果：

In [33]:
Person(name='Mike', gender='male')

Person(name='Mike', gender='male')

In [35]:
Person(name='Mike', gender='m')

ValueError: gender m is not valid

OK，结果显而易见了，第二个报错了，因为其值不是正常的性别，所以程序直接报错终止。

注意在 Validator 里面返回 True 或 False 是没用的，错误的值还会被照常复制。所以，一定要在 Validator 里面 raise 某个错误。

另外 attrs 库里面还给我们内置了好多 Validator，比如判断类型，这里我们再增加一个属性 age，必须为 int 类型：

```python
age = attrib(validator=validators.instance_of(int))
```


这时候初始化的时候就必须传入 int 类型，如果为其他类型，则直接抛错：

```python

TypeError: ("'age' must be <class 'int'> (got 'x' that is a <class 'str'>).
```

另外还有其他的一些 Validator，比如与或运算、可执行判断、可迭代判断等等，可以参考官方文档：https://www.attrs.org/en/stable/api.html#validators。



另外 validator 参数还支持多个 Validator，比如我们要设置既要是数字，又要小于 100，那么可以把几个 Validator 放到一个列表里面并传入：

In [36]:
from attr import attrs, attrib, validators

def is_less_than_100(instance, attribute, value):
    if value > 100:
        raise ValueError(f'age {value} must less than 100')

@attrs
class Person(object):
    name = attrib()
    gender = attrib(validator=is_valid_gender)
    age = attrib(validator=[validators.instance_of(int), is_less_than_100])

if __name__ == '__main__':
    print(Person(name='Mike', gender='male', age=500))

ValueError: age 500 must less than 100

## 转换器

其实很多时候我们会不小心传入一些形式不太标准的结果，比如本来是 int 类型的 100，我们传入了字符串类型的 100，那这时候直接抛错应该不好吧，所以我们可以设置一些转换器来增强容错机制，比如将字符串自动转为数字等等，看一个实例：

In [38]:
from attr import attrs, attrib

def to_int(value):
    try:
        return int(value)
    except:
        return None

@attrs
class Point(object):
    x = attrib(converter=to_int)
    y = attrib()

if __name__ == '__main__':
    print(Point('100', 3))

Point(x=100, y=3)


## 类型

为什么把这个放到最后来讲呢，因为 Python 中的类型是非常复杂的，有原生类型，有 typing 类型，有自定义类的类型。

首先我们来看看原生类型是怎样的，这个很容易理解了，就是普通的 int、float、str 等类型，其定义如下：

In [39]:
from attr import attrs, attrib

@attrs
class Point(object):
    x = attrib(type=int)
    y = attrib()

if __name__ == '__main__':
    print(Point(100, 3))
    print(Point('100', 3))

Point(x=100, y=3)
Point(x='100', y=3)


但我们发现，虽然定义了，但是不会被自动转类型的。

另外我们还可以自定义 typing 里面的类型，比如 List，另外 attrs 里面也提供了类型的定义：

In [40]:
from attr import attrs, attrib, Factory
import typing

@attrs
class Point(object):
    x = attrib(type=int)
    y = attrib(type=typing.List[int])
    z = attrib(type=Factory(list))

在这里我们定义了 Point 类代表离散点，随后定义了线，其拥有 points 属性是 Point 组成的列表。在初始化的时候我们声明了五个点，然后用这五个点组成的列表声明了一条线，逻辑没什么问题。

运行结果：

In [41]:
from attr import attrs, attrib, Factory
import typing

@attrs
class Point(object):
    x = attrib(type=int, default=0)
    y = attrib(type=int, default=0)

@attrs
class Line(object):
    name = attrib()
    points = attrib(type=typing.List[Point])

if __name__ == '__main__':
    points = [Point(i, i) for i in range(5)]
    print(points)
    line = Line(name='line1', points=points)
    print(line)

[Point(x=0, y=0), Point(x=1, y=1), Point(x=2, y=2), Point(x=3, y=3), Point(x=4, y=4)]
Line(name='line1', points=[Point(x=0, y=0), Point(x=1, y=1), Point(x=2, y=2), Point(x=3, y=3), Point(x=4, y=4)])


可以看到这里我们得到了一个嵌套类型的 Line 对象，其值是 Point 类型组成的列表。

以上便是一些属性的定义，把握好这些属性的定义，我们就可以非常方便地定义一个类了。

## 序列转换

在很多情况下，我们经常会遇到 JSON 等字符串序列和对象互相转换的需求，尤其是在写 REST API、数据库交互的时候。

attrs 库的存在让我们可以非常方便地定义 Python 类，但是它对于序列字符串的转换功能还是比较薄弱的，cattrs 这个库就是用来弥补这个缺陷的，下面我们再来看看 cattrs 这个库。

cattrs 导入的时候名字也不太一样，叫做 cattr，它里面提供了两个主要的方法，叫做 structure 和 unstructure，两个方法是相反的，对于类的序列化和反序列化支持非常好。

首先我们来看看基本的转换方法的用法，看一个基本的转换实例：